In [11]:
# Importing aux libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import copy

In [12]:
import gempy as gp
data_path = 'Data/'
save_path = 'Model_surface/'

In [35]:
geo_model = gp.create_model('Perarolo_3')

In [36]:
data_path = 'C:/Users/admin/Documents/Dottorato/Conda_envs/gempy_Rousseau/Data/'
# Importing the data from CSV-files and setting extent and resolution
gp.init_data(geo_model, [60, 220., -60., 60., 0, 140.], [50, 50, 50],
             path_i=data_path + "layer_i_nf.csv",
             path_o=data_path + "layer_o_nf.csv",
             default_values=True)

Active grids: ['regular']


Perarolo_3  2024-05-15 16:05

In [37]:
gp.map_stack_to_surfaces(geo_model,
                         {#"Fault_Series": 'Main_Fault',
                          "Strat_Series": ('gravely_layer', 'clay_layer',
                                           'altered_bedrock')},
                         remove_unused_series=True)
#"Fault_Series": 'Main_Fault',

,surface,series,order_surfaces,color,id
0,gravely_layer,Strat_Series,1,#015482,1
1,clay_layer,Strat_Series,2,#9f0052,2
2,altered_bedrock,Strat_Series,3,#ffbe00,3
3,basement,Basement,1,#728f02,4


In [38]:
gp.set_interpolator(geo_model,          
                    compile_theano=False,
                    theano_optimizer='fast_compile',
                    )

#with compile_theano=True it's really slow

Setting kriging parameters to their default values.
Compiling aesara function...
Level of Optimization:  fast_compile
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                       values
range             244.131112
$C_o$            1419.047619
drift equations       [3, 3]


In [9]:
# fp = data_path+ "topo_1_extrap.tif"
# fp

In [10]:

# geo_model.set_topography(source='gdal', filepath=fp)
# sol = gp.compute_model(geo_model, compute_mesh=True)

In [39]:
#NO TOPOGRAPHY
sol = gp.compute_model(geo_model, compute_mesh=True)

In [40]:
ver, sim = gp.get_surfaces(geo_model)
gpv = gp.plot_3d(geo_model, plotter_type='basic', show_surfaces=True, show_data=True, image=False,
                 show_topography=True)

c:\ProgramData\anaconda3\envs\gempy1\lib\site-packages\pyvista\utilities\helpers.py:507: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


In [42]:
geo_model.save_model(path = save_path, name='Model_surface')
geo_model.save_solution(path = save_path, name='Model_surface.npz')

In [27]:
geo_model = gp.load_model(path = save_path, name = 'Model_surface')

Active grids: ['regular']


In [43]:
#PROVA CON INDICI ORIGINALI ROUSSEAU
# create vertices and elements

path='FLAC3D_export/'
filename='geomodel_topo_11.f3grid'

def __build_vertices_elements_groups__(geo_model):

    # get model information
    nx, ny, nz = geo_model.grid.regular_grid.resolution
    xmin, xmax, ymin, ymax, zmin, zmax = geo_model.solutions.grid.regular_grid.extent

    # create vertices array
    dx, dy, dz = (xmax - xmin) / nx, (ymax - ymin) / ny, (zmax - zmin) / nz
    n_vertices = (nx + 1) * (ny + 1) * (nz + 1)
    vertices = np.zeros((n_vertices, 3), dtype='f8')
    vertices_ids = np.arange(n_vertices)  # used to generate coordinate
    vertices[:, 0] = vertices_ids % (nx + 1) * dx + xmin
    vertices[:, 1] = (vertices_ids % ((nx + 1) * (ny + 1))) // (nx + 1) * dy + ymin
    vertices[:, 2] = vertices_ids // ((nx + 1) * (ny + 1)) * dz + zmin

    # build elements
    n_elements = nx * ny * nz
    element_ids = np.arange(n_elements)  # used to generate elems
    elements = np.zeros((n_elements, 9), dtype='i8')
    i = element_ids % nz
    j = element_ids // nz % ny
    k = element_ids // (nz * ny)
    elements[:, 0] = 8  # all hex 
    #[1, 2, 3, 4, 5, 6, 7, 8]
    #[1, 2, 4, 5, 3, 8, 6, 7]
    elements[:, 1] = 1 + i * (nx + 1) * (ny + 1) + j * (nx + 1) + k #1
    elements[:, 2] = elements[:, 1] + 1                             #2
    elements[:, 3] = elements[:, 2] + (nx + 1)                      #4
    elements[:, 4] = elements[:, 3] - 1                             #5
    elements[:, 5] = elements[:, 1] + ((nx + 1) * (ny + 1))         #3
    elements[:, 6] = elements[:, 5] + 1                             #8
    elements[:, 7] = elements[:, 6] + (nx + 1)                      #6
    elements[:, 8] = elements[:, 7] - 1                             #7
    
    # build groups
    lith_ids = np.round(geo_model.solutions.lith_block)
    lith_ids = lith_ids.astype(int)
    sids = dict(zip(geo_model._surfaces.df['surface'], geo_model._surfaces.df['id']))
    groups = {}
    for region_name, region_id in sids.items():
        cell_ids = np.where(lith_ids == region_id)[0] + 1
        if not len(cell_ids): continue
        groups[region_name] = cell_ids

    B8_index = []
    W6_index = []
    new_el = copy.deepcopy(elements)
    new_groups = copy.deepcopy(groups)

    for x in range(0,nz*ny*nx,nz*ny):
        for y in range(x,x+nz*ny,nz):
            for z in range(y,y+nz,1):
                if z < y+nz-1:
                    if lith_ids[z]!=lith_ids[z+1]:
                        #roof.update({z:[lith_ids[z], lith_ids[z+1]]})
                        new_row = new_el[z,:]
                        new_el[z,8] = new_el[z,3]
                        new_el[z,6] = new_el[z,7]
                        new_row[1] = new_row[2]
                        new_row[5] = new_row[3]
                        new_el = np.vstack((new_el, new_row))
                        l = len(new_el)-1
                        for region_name, region_id in sids.items():
                            if region_id == lith_ids[z+1]:
                                aux = new_groups[region_name]
                                aux = np.append(aux, l)
                                new_groups[region_name] = aux
                        B8_index.append(z)
                        W6_index.append(l)

    # remove element above topography
    #mask_topo = geo_model._grid.regular_grid.mask_topo
    shape = geo_model._grid.regular_grid.resolution
    shape_tot = shape[0]*shape[1]*shape[2]
    mask_topo = geo_model._grid.regular_grid.mask_topo
    if(mask_topo.size > 0):
        inactive_cells = mask_topo.reshape(shape_tot)
    else:
        inactive_cells = None
    if np.any(inactive_cells):
        # update correspondance
        new_id_vertices = np.zeros(len(vertices), dtype='i8')
        new_id_elements = np.zeros(len(elements), dtype='i8')
        # remove inactive cell
        elements = elements[~inactive_cells]
        new_id_elements[~inactive_cells] = np.arange(len(elements)) + 1
        # remove deleted vertices
        cond = np.isin(np.arange(len(vertices)) + 1, elements[:, 1:].flatten())
        vertices = vertices[cond]
        new_id_vertices[cond] = np.arange(len(vertices)) + 1
        # renumber vertices in element
        elements[1:] = new_id_vertices[elements[1:] - 1]
        # renumber groups
        for grp in groups.values():
            grp[:] = new_id_elements[grp - 1]

    return vertices, new_el, new_groups, B8_index, W6_index


vertices, elements, groups, B8, W6 = __build_vertices_elements_groups__(geo_model)

# open output file
# if not path:
#    path = './'
# if not os.path.exists(path):
#    os.makedirs(path)

out = open(path + filename, 'w')

# write gridpoints
out.write("*GRIDPOINTS")
for i, vertice in enumerate(vertices):
    out.write(f"\nG {i + 1} {vertice[0]} {vertice[1]} {vertice[2]}")

# write elements
out.write('\n*ZONES')
zone_counter = 1 

new_column_order = [1, 2, 4, 5, 3, 8, 6, 7]

for i, elem in enumerate(elements[B8]):
    out.write(f'\nZ B8 {zone_counter}')  
    for x in new_column_order:
        out.write(f" {elem[x]}")
    zone_counter += 1 

for i, elem in enumerate(elements[W6]):
    out.write(f'\nZ W6 {zone_counter}')  
    for x in new_column_order:
        out.write(f" {elem[x]}")
    zone_counter += 1 

# make groups
out.write('\n*GROUPS\n')
for grp_name, grp in groups.items():
    out.write(f'ZGROUP \"{grp_name}\"\n')
    count = 0
    for x in grp:
        out.write(f"{x} ")
        count += 1
        if count == 8:
            out.write("\n")
            count = 0
    if count != 0: out.write("\n")

out.close()
print("Successfully exported geological model as FLAC3D input to " + path)

Successfully exported geological model as FLAC3D input to FLAC3D_export/
